<a href="https://colab.research.google.com/github/filipecalegario/stylegan2-ada-experiments/blob/main/Treinamento_StyleGan2_ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA (PyTorch): Treinando um modelo a partir de imagens pré-processadas

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch

## Configurações Iniciais

**Dica**: Caso seu modelo demore algumas horas para ser treinado (provavelmente levará), é normal que o Colab te desconecte durante o treinamento algumas vezes. Usando o Colab Pro, a frequência dessas desconexões será bastante reduzida. Caso ainda prefira utilizar o Colab Free, dê uma olhada nesse artigo para diminuir a frequência das desconexões: 
<br>
https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

### Checar uso de GPU (placa de vídeo)

Ao executar a célula a seguir, é esperado que você veja uma tabela com as configurações da placa de vídeo atualmente utilizada por seu ambiente de execução.
<br>
Caso o comando a seguir falhe, vá no menu do canto superior esquerdo e, na aba ***Runtime***, entre em ***Change runtime type***. Dentro desse novo menu, vá em ***Hardware accelerator*** e selecione a opção ***GPU***, por fim salve.
<br>
Rode novamente a célula abaixo para checar se o ambiente de execução agora tem uma placa de vídeo conectada.

In [ ]:
!nvidia-smi

### Instalação de pré-requisitos

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch/issues/2

In [ ]:
%pip install ninja

### Baixar código do StyleGAN2-ADA (PyTorch)

Referência: https://github.com/woctezuma/stylegan2-ada-pytorch/tree/google-colab 
<br>
Fork utilizada: https://github.com/ClaudioCarvalhoo/stylegan2-ada-pytorch

In [ ]:
%cd /content/

%rm -rf stylegan2-ada-pytorch/
!git clone https://github.com/ClaudioCarvalhoo/stylegan2-ada-pytorch

%cd stylegan2-ada-pytorch/

%cd /content/

## Conectar com Google Drive

In [ ]:
%pip install Google-Colab-Transfer

Ao executar a próxima célula, você receberá um link para pegar seu código de verificação do Google Drive.
<br>
Acesse com sua conta que tem as imagens de treinamento no Drive e cole o código recebido na caixa de texto que aparecerá abaixo.

In [ ]:
import colab_transfer

colab_transfer.mount_google_drive()

Após executar a próxima célula, insira nas caixas de texto o caminho para a pasta do seu Drive que contém as imagens pré-processadas anteriormente geradas e também o caminho para a pasta do seu Drive na qual deseja salvar seu modelo.
<br>
Lembre-se de manter intacta a parte inicial do texto (/content/drive/MyDrive/) pois é o caminho do Colab para acessar a raíz de seu Drive.

In [ ]:
import ipywidgets as widgets

dataset = widgets.Text(
    value='/content/drive/MyDrive/sua_pasta/dataset',
    placeholder='Digite ou cole aqui o caminho',
    layout=widgets.Layout(width='50%')
)
output_folder = widgets.Text(
    value='/content/drive/MyDrive/sua_pasta/modelos',
    placeholder='Digite ou cole aqui o caminho',
    layout=widgets.Layout(width='50%')
)
widgets.VBox([
    widgets.VBox([widgets.Label('Caminho para as imagens processadas:'), dataset]),          
    widgets.VBox([widgets.Label('Caminho para salvar o modelo:'), output_folder])
])

## Treinando sua rede

Referência: https://github.com/NVlabs/stylegan2-ada-pytorch#training-new-networks

### Executando o treinamento

Uma vez que o treinamento for iniciado, ele criará modelos intermediários e salvará eles no seu Drive como arquivos que têm a extensão *.pkl*. Como normalmente não conseguimos realizar o treinamento completo em uma única execução (pois o Colab desconecta depois de um tempo), utilizaremos o mais recente destes modelos intermediários para retormar nosso treinamento quando nos reconectarmos.<br><br>

*   Primeira Execução

Coloque o valor **noresume** na caixa de texto abaixo para começar o treinamento do zero.

*   Retomando a Execução

Coloque o caminho para o último modelo (arquivo *.pkl*) criado em seu Drive, para que possamos retomar a partir dele.

In [ ]:
resume = widgets.Text(
    value='noresume',
    placeholder='Insira aqui o caminho para o modelo',
    layout=widgets.Layout(width='50%')
)
    
widgets.VBox([widgets.Label('Caminho para retomar treinamento:'), resume]) 

A célula abaixo contém alguns parâmetros para o treinamento de seu modelo, que podem ser ajustados caso deseje estudar sobre os efeitos de cada um.
<br>
Atenção para o parâmetro "mini_batch". Caso ao rodar o treinamento você receba um erro alegando falta de memória, experimente diminuir o mini_batch e rodar novamente a célula abaixo. Normalmente o valor 16 já resolve a maioria dos casos, mas tente usar 32 antes se conseguir sem erros pois terá maior eficiência.

In [ ]:
snap = 5
gamma = 10
kimg = 100000
mini_batch = 32

Ao rodar a célula abaixo, você iniciará/retomará seu treinamento. Isso pode potencialmente durar horas, dias ou até mesmo semanas.
<br>
Lembre-se que não há um momento final no qual seu modelo está "pronto". Você deve acompanhar a evolução de seus modelos intermediários à medida que eles forem sendo gerados na pasta que você definiu como destino. Cada modelo será gerado com uma imagem que contém alguns exemplares do que ele é capaz de produzir. Utilize essas imagens para definir quando você considera que tem um modelo que produz bons resultados, nesse momento você poderá parar o treinamento e usar este modelo para o que desejar no futuro.

In [ ]:
!python stylegan2-ada-pytorch/train.py \
 --outdir={output_folder.value} \
 --snap={snap} \
 --metrics=none \
 --data={dataset.value} \
 --cfg=auto_norp \
 --cifar_tune=1 \
 --gamma={gamma} \
 --kimg={kimg} \
 --batch={mini_batch} \
 --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume={resume.value} \
